<a href="https://colab.research.google.com/github/cohere-ai/sandbox-topically/blob/main/notebooks/Intro%20-%20Topically%20with%20BERTopic.ipynb" target="_parent\"><img src="https://colab.research.google.com/assets/colab-badge.svg\" alt="Open In Colab"/></a>

In [ ]:
!pip install bertopic topically

In [14]:
from topically import Topically
from bertopic import BERTopic
import numpy as np
import pandas as pd

In [2]:
!wget https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_embeds.npy
# or
# !curl https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_embeds.npy -o askhn3k_embeds.npy

zsh:1: command not found: wget


In [15]:
#Load the embeddings matrix
embeds = np.load('askhn3k_embeds.npy')

# Load the dataframe containing the text and metadata of the posts
df = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)

print(f'Loaded a DataFrame with {len(df)} rows and an embeddings matrix of dimensions {embeds.shape}')

Loaded a DataFrame with 3000 rows and an embeddings matrix of dimensions (3000, 1024)


In [16]:
# Load and initialize BERTopic. Limit to 10 topics only.
topic_model = BERTopic(nr_topics=10) 

# df is a dataframe. df['title'] is the column of text we're modeling
df['topic'], probabilities = topic_model.fit_transform(df['title'], embeds)

In [17]:
# Pass in Cohere API key, or it will ask for it
app = Topically()

Enter your Cohere API Key········


In [23]:
# Name each cluster. This will make one request to GENERATE for each cluster.
# Since we have 10 clusters, this will call Cohere Generate 10 times.
df['cluster_names'] = app.name_clusters((df['title'], df['topic']))

In [25]:
# Preview the generated names
df[['title', 'topic', 'cluster_names']]

,title,topic,cluster_names
0,"I'm a software engineer going blind, how shoul...",-1,Best Practices for Technical Talks
1,Am I the longest-serving programmer – 57 years...,4,How to become a better programmer?
2,Is S3 down?,-1,Best Practices for Technical Talks
3,What tech job would let me get away with the l...,-1,Best Practices for Technical Talks
4,What books changed the way you think about alm...,3,Programming and computer science
...,...,...,...
2995,Do you like/use Reddit's redesign?,-1,Best Practices for Technical Talks
2996,What's your best startup idea that you're not ...,-1,Best Practices for Technical Talks
2997,Freelancer? Seeking freelancer? (June 2021),9,Freelance and contract
2998,How to learn about the history of computing?,7,How to learn math


We can run name_clusters again to get new names. There are several ways (more on the way) to improve the generated titles:

1. Make a custom prompt specific to the dataset. Use the prompts in prompts/prompts.py as a starting point.
1. Pass `num_generations=5` to `name_clusters()`. This will tell Cohere Generate to return five suggested titles for each topic, and the one with the highest likelihood would be selected as the topic.
1. [coming soon] Include cTFIDF keywords in the prompt